# Importing required Libraries

In [1]:
import pandas as pd
from pulp import *
from num2words import *

# Data Preprocessing

In [2]:
inventory0 = pd.read_csv('inventory.csv')
sales0 = pd.read_csv('Weekly-Sales.csv')
productdesc0 = pd.read_csv('products.csv').set_index('Product_ID')
stores0 = pd.read_csv('stores.csv')

In [3]:
# Keeping More weeks will require more computation time
week = [None]*6
for i in range(0,6):
    week[i] =i

In [4]:
stores = [None]*50
for i in range(0,50):
    stores[i] = i+1
products = [None]*35
for i in range(0,35):
    products[i]=i+1

In [5]:
sales = {}
for s in stores:
    for p in products:
        for w in week:
            sales[(s,p,w)]=(sales0[(sales0['Store ID']==s)&(sales0['Product ID']==p)&(sales0['Week']==w)&(sales0['Year of Date']==2017)]['Count of sales'])
            if len(sales[s,p,w]>0):
                sales[s,p,w]=int(sales[s,p,w])
            else:
                sales[s,p,w]=0
del sales0     

In [6]:
openinginventory = {}
for s in stores:
    for p in products:
        openinginventory[(s,p)]=inventory0[(inventory0['Store_ID']==s)&(inventory0['Product_ID']==p)]['Stock_On_Hand']
        if len(openinginventory[(s,p)]>0):
            openinginventory[(s,p)] = int(openinginventory[(s,p)])
        else:
            openinginventory[(s,p)] = 0
del inventory0

In [7]:
Pcost = {}
Pprice = {}
Pprofit = {}
for p in products:
    Pcost[p]=float(productdesc0.loc[p]['Product_Cost'][1:5])
    Pprice[p]=float(productdesc0.loc[p]['Product_Price'][1:5])
    Pprofit[p] = Pprice[p] - Pcost[p]


# Model

## Declaring Objective Variable

In [8]:
# Objective Variable
z = LpProblem('Maximize_the_profit',LpMaximize)

## Decision Variables

In [9]:
# Decision Variable
y = LpVariable.dict('Selling',(stores,products,week),lowBound=0,cat='Integer')
capital_unused = LpVariable.dict('Selling',(week),lowBound=0,cat='Continous')

## Objective Function

In [10]:
# Objective Function
# Sell products with maximum cost to profit ratio
z += lpSum(y[s,p,w]*Pprofit[p] for s in stores for p in products for w in week) - lpSum(y[s,p,0]*Pprofit[p] for s in stores for p in products)

## Constraints

### Constraints related to capital

In [11]:
for w in range(1,6):
    z += lpSum(y[s,p,w]*Pcost[p] for s in stores for p in products) + capital_unused[w] == lpSum(y[s,p,w-1]*Pprice[p] for s in stores for p in products)
z += lpSum(y[s,p,0]*Pprice[p] for s in stores for p in products) == 50000
for s in stores:
    for p in products:
        for w in range(1,6):
                z += y[s,p,w] <= sales[s,p,w]

### Running our model

In [12]:
status = z.solve()
total_profit = value(z.objective)
print("Solution is :",LpStatus[status])
print("The total profit is :",total_profit)
print("Total Profit in Words :",(num2words(total_profit,lang='en_IN')))

Solution is : Optimal
The total profit is : 137149.27000000008
Total Profit in Words : one lakh, thirty-seven thousand, one hundred and forty-nine point two seven zero zero zero zero zero zero zero zero seven


### Checking for any disperancy 

In [13]:
for w in range(1,6):
    for s in stores:
        for p in products:
            if (y[s,p,w].varValue > sales[(s,p,w)]):
                print(y[s,p,w],":",y[s,p,w].varValue,"::",sales[(s,p,w)])


Selling_21_15_0 : 8195.0 :: 0
Selling_27_25_0 : 305.0 :: 0


## Analyzing our Result

In [14]:
for w in range(1,6) :
    print("\nCapital Available in week",w,":",lpSum(y[s,p,w-1].varValue*Pprice[p] for s in stores for p in products))
    print("Capital Required in week",w,":",lpSum(sales[s,p,w]*Pcost[p] for s in stores for p in products))
    print("Capital Used in week",w,":",lpSum(y[s,p,w].varValue*Pcost[p] for s in stores for p in products))
    print("Capital Not Used in week",w,":",capital_unused[w].varValue)
    print("Revenue of week",w,":",lpSum(y[s,p,w].varValue*Pprice[p] for s in stores for p in products))
    print("Profit of week",w,":",lpSum(y[s,p,w].varValue*Pprofit[p] for s in stores for p in products))

    


Capital Available in week 1 : 50000.0
Capital Required in week 1 : 73571.75
Capital Used in week 1 : 49997.59999999991
Capital Not Used in week 1 : 2.4
Revenue of week 1 : 76290.84999999995
Profit of week 1 : 26293.249999999985

Capital Available in week 2 : 76290.84999999995
Capital Required in week 2 : 61901.439999999966
Capital Used in week 2 : 61901.439999999966
Capital Not Used in week 2 : 14389.41
Revenue of week 2 : 87821.92000000004
Profit of week 2 : 25920.479999999992

Capital Available in week 3 : 87821.92000000004
Capital Required in week 3 : 66469.01000000005
Capital Used in week 3 : 66469.01000000005
Capital Not Used in week 3 : 21352.91
Revenue of week 3 : 93753.32999999999
Profit of week 3 : 27284.31999999999

Capital Available in week 4 : 93753.32999999999
Capital Required in week 4 : 69357.11999999997
Capital Used in week 4 : 69357.11999999997
Capital Not Used in week 4 : 24396.21
Revenue of week 4 : 97753.02999999993
Profit of week 4 : 28395.90999999998

Capital Ava